In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@5b5381d6

In [2]:
import spark.implicits._

case class Product(product_id: Int, product_name: String, product_category: String)
case class Order(product_id: Int, order_date: String, unit: Int)

val product: Seq[(Int, String, String)] = Seq(
  (1, "Leetcode Solutions", "Book"),
  (2, "Jewels of Stringology", "Book"),
  (3, "HP", "Laptop"),
  (4, "Lenovo", "Laptop"),
  (5, "Leetcode Kit", "T-shirt")
)

val order: Seq[(Int, String, Int)] = Seq(
  (1, "2020-02-05", 60),
  (1, "2020-02-10", 70),
  (2, "2020-01-18", 30),
  (2, "2020-02-11", 80),
  (3, "2020-02-17", 2),
  (3, "2020-02-24", 3),
  (4, "2020-03-01", 20),
  (4, "2020-03-04", 30),
  (4, "2020-03-04", 60),
  (5, "2020-02-25", 50),
  (5, "2020-02-27", 50),
  (5, "2020-03-01", 50)
)

val productDS = product.toDF("product_id", "product_name", "product_category").as[Product]
val orderDS = order.toDF("product_id", "order_date", "unit").as[Order]

import spark.implicits._


defined class Product
defined class Order
product: Seq[(Int, String, String)] = List(
  (1, "Leetcode Solutions", "Book"),
  (2, "Jewels of Stringology", "Book"),
  (3, "HP", "Laptop"),
  (4, "Lenovo", "Laptop"),
  (5, "Leetcode Kit", "T-shirt")
)
order: Seq[(Int, String, Int)] = List(
  (1, "2020-02-05", 60),
  (1, "2020-02-10", 70),
  (2, "2020-01-18", 30),
  (2, "2020-02-11", 80),
  (3, "2020-02-17", 2),
  (3, "2020-02-24", 3),
  (4, "2020-03-01", 20),
  (4, "2020-03-04", 30),
  (4, "2020-03-04", 60),
  (5, "2020-02-25", 50),
  (5, "2020-02-27", 50),
  (5, "2020-03-01", 50)
)
productDS: Dataset[Product] = [product_id: int, product_name: string ... 1 more field]
orderDS: Dataset[Order] = [product_id: int, order_date: string ... 1 more field]

Write a solution to get the names of products that have at least 100 units ordered in February 2020 and their amount.

Return the result table in any order.

The result format is in the following example.

| product_name       | unit    |
|--------------------|---------|
| Leetcode Solutions | 130     |
| Leetcode Kit       | 100     |

In [3]:
orderDS.select(
        col("product_id"),
        date_trunc("month", col("order_date")).as("order_month"),
        col("unit")
    )
    .filter(col("order_month") === to_date(lit("2020-02-01")))
    .groupBy("product_id")
    .agg(sum(col("unit")).as("unit"))
    .filter(col("unit") >= lit(100))
    .join(productDS, Seq("product_id"), "left_outer")
    .select(col("product_name"), col("unit"))
    .show()

+------------------+----+
|      product_name|unit|
+------------------+----+
|Leetcode Solutions| 130|
|      Leetcode Kit| 100|
+------------------+----+



In [4]:
spark.stop()